In [1]:
from shroomdk import ShroomDK

# Initialize `ShroomDK` with your API Key
sdk = ShroomDK("ce7ee339-c712-4e83-a06b-6039a058ca43")

In [39]:
from datetime import datetime
import pandas as pd

# DAILY QUERY RUN LIMIT 10000
# BURST QUERY RUN LIMIT 250 (maybe 5miniutes?)
# DATE_FORMAT ='%Y-%m-%d %H:%M:%S'

dex_list = { 
    'ether' : {
        'uniswap_v3' : 'ethereum.uniswapv3.ez_swaps' 
    }
}
df_list = {}


def querying_pagination(query_string):
    result_list = []
    for i in range(1,11): # max is a million rows @ 100k per page
        data=sdk.query(query_string,page_size=100000,page_number=i)
        if data.run_stats.record_count == 0:  
            break
        else:
            result_list.append(data.records)
        
    # Loops through the returned results and adds into a pandas dataframe
    result_df=pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df=pd.json_normalize(each_list)
        else:
            result_df=pd.concat([result_df, pd.json_normalize(each_list)])

    return result_df



def querying_swap(start_date,end_date,chain='ether',dex='uniswap_v3'):
    sql = f"""
        SELECT * 
        FROM {dex_list[chain][dex]}
        WHERE BLOCK_TIMESTAMP >= '{start_date}' AND BLOCK_TIMESTAMP <= '{end_date}'
        ORDER BY BLOCK_TIMESTAMP
    """
    df = querying_pagination(sql)
    if chain not in df_list:
        df_list[chain]={}
    if dex in df_list[chain]:
        df_list[chain][dex] = pd.concat([df_list[chain][dex], df])
        df_list[chain][dex].drop_duplicates(inplace=True)
        df_list[chain][dex].sort_values('block_timestamp')
        df_list[chain][dex].reset_index(drop=True,inplace=True)
    else:
        df_list[chain][dex]=df        
            


In [43]:
querying_swap(start_date = '2023-04-03 18:00:11',end_date = '2023-04-03 19:00:11')

In [47]:
df_list['ether']['uniswap_v3'].groupby(['block_number','token0_symbol','token1_symbol']).count()['blockchain']

block_number  token0_symbol  token1_symbol
16970129      BLUR           USDC             1
                             WETH             1
              SOV            WETH             2
              TRG            WETH             1
              USDC           WETH             1
                                             ..
16970426      UNI            WETH             1
              USDC           USDT             1
                             WETH             1
              WETH           FUND             1
                             USDT             3
Name: blockchain, Length: 1861, dtype: int64